# Package for WRC Results Scrape

This notebook contains most of the ingredients for a package that can be used to scrape results data from current and archived WRC Live Timing pages.

At the moment, the script will only happily downloaded the complete set of results for a particular rally. The code needs updating to allow:

- loading data for a completed stage into a database that already contains results for other completed stages;
- incremental loads of data to upsert data into the database from an ongoing stage;

In [ ]:
#https://www.wrc.com/service/sasCacheApi.php?route=rallies%2F40%2Fitinerary

In [72]:
YEAR = 2019

In [270]:
url_root='http://www.wrc.com/service/sasCacheApi.php?route={stub}'
url_base='http://www.wrc.com/service/sasCacheApi.php?route=events/78/{stub}'

In [2]:
#Call a resource by ID
wrcapi='https://webappsdata.wrc.com/srv/wrc/json/api/wrcsrv/byId?id=%22{}%22' #requires resource ID

In [272]:
stubs = { 'itinerary': 'rallies/{rallyId}/itinerary',
          'startlists': 'rallies/{rallyId}/entries',
         'penalties': 'rallies/{rallyId}/penalties',
         'retirements': 'rallies/{rallyId}/retirements',
         'stagewinners':'rallies/{rallyId}/stagewinners',
         'overall':'stages/{stageId}/results?rallyId={rallyId}',
         'split_times':'stages/{stageId}/splittimes?rallyId={rallyId}',
         'stage_times_stage':'stages/{stageId}/stagetimes?rallyId={rallyId}',
         'stage_times_overall':'stages/{stageId}/results?rallyId={rallyId}',
         'seasons':'seasons',
         'seasonDetails':'seasons/{seasonId}',
         # TO DO - for 2019, the following may need to be prefixed with seasons/ ?
         'championship':'seasons/4/championships/{championshipId}',
         'championship_results':'seasons/4/championships/{championshipId}/results',
        }

In [205]:
#pip3 install sqlite-utils

import requests
import re
import json
from bs4 import BeautifulSoup 

import sqlite3
from sqlite_utils import Database

import pandas as pd
from pandas.io.json import json_normalize

#!pip3 install isodate
import isodate

## Database Schema

The following tables are literal mappings from flattened JSON datafiles published by the WRC.

The data model (primary and foreign key relationships) is derived by observation.

In [321]:
#SQL in wrcResults.sql
setup_q='''
CREATE TABLE "itinerary_event" (
  "eventId" INTEGER,
  "itineraryId" INTEGER PRIMARY KEY,
  "name" TEXT,
  "priority" INTEGER
);
CREATE TABLE "itinerary_legs" (
  "itineraryId" INTEGER,
  "itineraryLegId" INTEGER PRIMARY KEY,
  "legDate" TEXT,
  "name" TEXT,
  "order" INTEGER,
  "startListId" INTEGER,
  "status" TEXT,
  FOREIGN KEY ("itineraryId") REFERENCES "itinerary_event" ("itineraryId")
);
CREATE TABLE "itinerary_sections" (
  "itineraryLegId" INTEGER,
  "itinerarySectionId" INTEGER PRIMARY KEY,
  "name" TEXT,
  "order" INTEGER,
  FOREIGN KEY ("itineraryLegId") REFERENCES "itinerary_legs" ("itineraryLegId")
);
CREATE TABLE "itinerary_stages" (
  "code" TEXT,
  "distance" REAL,
  "eventId" INTEGER,
  "name" TEXT,
  "number" INTEGER,
  "stageId" INTEGER PRIMARY KEY,
  "stageType" TEXT,
  "status" TEXT,
  "timingPrecision" TEXT,
  "itineraryLegId" INTEGER,
  "itinerarySections.itinerarySectionId" INTEGER,
  FOREIGN KEY ("itineraryLegId") REFERENCES "itinerary_legs" ("itineraryLegId")
);
CREATE TABLE "itinerary_controls" (
  "code" TEXT,
  "controlId" INTEGER PRIMARY KEY,
  "controlPenalties" TEXT,
  "distance" REAL,
  "eventId" INTEGER,
  "firstCarDueDateTime" TEXT,
  "firstCarDueDateTimeLocal" TEXT,
  "location" TEXT,
  "stageId" INTEGER,
  "status" TEXT,
  "targetDuration" TEXT,
  "targetDurationMs" INTEGER,
  "timingPrecision" TEXT,
  "type" TEXT,
  "itineraryLegId" INTEGER,
  "itinerarySections.itinerarySectionId" INTEGER,
  "roundingPolicy" TEXT,
  FOREIGN KEY ("itineraryLegId") REFERENCES "itinerary_legs" ("itineraryLegId")
);
CREATE TABLE "startlists" (
  "codriver.abbvName" TEXT,
  "codriver.code" TEXT,
  "codriver.country.countryId" INTEGER,
  "codriver.country.iso2" TEXT,
  "codriver.country.iso3" TEXT,
  "codriver.country.name" TEXT,
  "codriver.countryId" INTEGER,
  "codriver.firstName" TEXT,
  "codriver.fullName" TEXT,
  "codriver.lastName" TEXT,
  "codriver.personId" INTEGER,
  "codriverId" INTEGER,
  "driver.abbvName" TEXT,
  "driver.code" TEXT,
  "driver.country.countryId" INTEGER,
  "driver.country.iso2" TEXT,
  "driver.country.iso3" TEXT,
  "driver.country.name" TEXT,
  "driver.countryId" INTEGER,
  "driver.firstName" TEXT,
  "driver.fullName" TEXT,
  "driver.lastName" TEXT,
  "driver.personId" INTEGER,
  "driverId" INTEGER,
  "eligibility" TEXT,
  "entrant.entrantId" INTEGER,
  "entrant.logoFilename" TEXT,
  "entrant.name" TEXT,
  "entrantId" INTEGER,
  "entryId" INTEGER PRIMARY KEY,
  "eventId" INTEGER,
  "group.name" TEXT,
  "groupId" INTEGER,
  "group.groupId" INTEGER,
  "identifier" TEXT,
  "manufacturer.logoFilename" TEXT,
  "manufacturer.manufacturerId" INTEGER,
  "manufacturer.name" TEXT,
  "manufacturerId" INTEGER,
  "priority" TEXT,
  "status" TEXT,
  "tag" TEXT,
  "tag.name" TEXT,
  "tag.tagId" INTEGER,
  "tagId" INTEGER,
  "tyreManufacturer" TEXT,
  "vehicleModel" TEXT,
  "entryListOrder" INTEGER,
  FOREIGN KEY ("eventId") REFERENCES "itinerary_event" ("eventId")
);
CREATE TABLE "roster" (
  "fiasn" INTEGER,
  "code" TEXT,
  "sas-entryid" INTEGER PRIMARY KEY,
  "roster_num" INTEGER,
  FOREIGN KEY ("sas-entryid") REFERENCES "startlists" ("entryId")
);
CREATE TABLE "startlist_classes" (
  "eventClassId" INTEGER,
  "eventId" INTEGER,
  "name" TEXT,
  "entryId" INTEGER,
  PRIMARY KEY ("eventClassId","entryId"),
  FOREIGN KEY ("eventId") REFERENCES "itinerary_event" ("eventId"),
  FOREIGN KEY ("entryId") REFERENCES "startlists" ("entryId")
);
CREATE TABLE "penalties" (
  "controlId" INTEGER,
  "entryId" INTEGER,
  "penaltyDuration" TEXT,
  "penaltyDurationMs" INTEGER,
  "penaltyId" INTEGER PRIMARY KEY,
  "reason" TEXT,
  FOREIGN KEY ("entryId") REFERENCES "startlists" ("entryId")
);
CREATE TABLE "retirements" (
  "controlId" INTEGER,
  "entryId" INTEGER,
  "reason" TEXT,
  "retirementDateTime" TEXT,
  "retirementDateTimeLocal" TEXT,
  "retirementId" INTEGER PRIMARY KEY,
  "status" TEXT,
  FOREIGN KEY ("entryId") REFERENCES "startlists" ("entryId")
);
CREATE TABLE "stagewinners" (
  "elapsedDuration" TEXT,
  "elapsedDurationMs" INTEGER,
  "entryId" INTEGER,
  "stageId" INTEGER,
  "stageName" TEXT,
  PRIMARY KEY ("stageId"),
  FOREIGN KEY ("entryId") REFERENCES "startlists" ("entryId"),
  FOREIGN KEY ("stageId") REFERENCES "itinerary_stages" ("stageId")
);
CREATE TABLE "stage_overall" (
  "diffFirst" TEXT,
  "diffFirstMs" INTEGER,
  "diffPrev" TEXT,
  "diffPrevMs" INTEGER,
  "entryId" INTEGER,
  "penaltyTime" TEXT,
  "penaltyTimeMs" INTEGER,
  "position" INTEGER,
  "stageTime" TEXT,
  "stageTimeMs" INTEGER,
  "totalTime" TEXT,
  "totalTimeMs" INTEGER,
  "stageId" INTEGER,
  PRIMARY KEY ("stageId","entryId"),
  FOREIGN KEY ("stageId") REFERENCES "itinerary_stages" ("stageId"),
  FOREIGN KEY ("entryId") REFERENCES "startlists" ("entryId")
);
CREATE TABLE "split_times" (
  "elapsedDuration" TEXT,
  "elapsedDurationMs" INTEGER,
  "entryId" INTEGER,
  "splitDateTime" TEXT,
  "splitDateTimeLocal" TEXT,
  "splitPointId" INTEGER,
  "splitPointTimeId" INTEGER PRIMARY KEY,
  "stageTimeDuration" TEXT,
  "stageTimeDurationMs" REAL,
  "startDateTime" TEXT,
  "startDateTimeLocal" TEXT,
  "stageId" INTEGER,
  FOREIGN KEY ("stageId") REFERENCES "itinerary_stages" ("stageId"),
  FOREIGN KEY ("entryId") REFERENCES "startlists" ("entryId")
);
CREATE TABLE "stage_times_stage" (
  "diffFirst" TEXT,
  "diffFirstMs" INTEGER,
  "diffPrev" TEXT,
  "diffPrevMs" INTEGER,
  "elapsedDuration" TEXT,
  "elapsedDurationMs" INTEGER,
  "entryId" INTEGER,
  "position" INTEGER,
  "source" TEXT,
  "stageId" INTEGER,
  "stageTimeId" INTEGER PRIMARY KEY,
  "status" TEXT,
  FOREIGN KEY ("stageId") REFERENCES "itinerary_stages" ("stageId"),
  FOREIGN KEY ("entryId") REFERENCES "startlists" ("entryId")
);
CREATE TABLE "stage_times_overall" (
  "diffFirst" TEXT,
  "diffFirstMs" INTEGER,
  "diffPrev" TEXT,
  "diffPrevMs" INTEGER,
  "entryId" INTEGER,
  "penaltyTime" TEXT,
  "penaltyTimeMs" INTEGER,
  "position" INTEGER,
  "stageTime" TEXT,
  "stageTimeMs" INTEGER,
  "totalTime" TEXT,
  "totalTimeMs" INTEGER,
  "stageId" INTEGER,
  PRIMARY KEY ("stageId","entryId"),
  FOREIGN KEY ("stageId") REFERENCES "itinerary_stages" ("stageId"),
  FOREIGN KEY ("entryId") REFERENCES "startlists" ("entryId")
);
CREATE TABLE "championship_lookup" (
  "championshipId" INTEGER PRIMARY KEY,
  "fieldFiveDescription" TEXT,
  "fieldFourDescription" TEXT,
  "fieldOneDescription" TEXT,
  "fieldThreeDescription" TEXT,
  "fieldTwoDescription" TEXT,
  "name" TEXT,
  "seasonId" INTEGER,
  "type" TEXT,
  "_codeClass" TEXT,
  "_codeTyp" TEXT
);
CREATE TABLE "championship_results" (
  "championshipEntryId" INTEGER,
  "championshipId" INTEGER,
  "dropped" INTEGER,
  "eventId" INTEGER,
  "pointsBreakdown" TEXT,
  "position" INTEGER,
  "publishedStatus" TEXT,
  "status" TEXT,
  "totalPoints" INTEGER,
  PRIMARY KEY ("championshipEntryId","eventId"),
  FOREIGN KEY ("championshipId") REFERENCES "championship_lookup" ("championshipId"),
  FOREIGN KEY ("eventId") REFERENCES "itinerary_event" ("eventId")
);
CREATE TABLE "championship_entries_codrivers" (
  "championshipEntryId" INTEGER PRIMARY KEY,
  "championshipId" INTEGER,
  "entrantId" TEXT,
  "ManufacturerTyre" TEXT,
  "Manufacturer" TEXT,
  "FirstName" TEXT,
  "CountryISO3" TEXT,
  "CountryISO2" TEXT,
  "LastName" TEXT,
  "manufacturerId" INTEGER,
  "personId" INTEGER,
  "tyreManufacturer" TEXT,
  FOREIGN KEY ("championshipId") REFERENCES "championship_lookup" ("championshipId")
);
CREATE TABLE "championship_entries_manufacturers" (
  "championshipEntryId" INTEGER PRIMARY KEY ,
  "championshipId" INTEGER,
  "entrantId" INTEGER,
  "Name" TEXT,
  "LogoFileName" TEXT,
  "Manufacturer" TEXT,
  "manufacturerId" INTEGER,
  "personId" TEXT,
  "tyreManufacturer" TEXT,
  FOREIGN KEY ("championshipId") REFERENCES "championship_lookup" ("championshipId")
);
CREATE TABLE "championship_rounds" (
  "championshipId" INTEGER,
  "eventId" INTEGER,
  "order" INTEGER,
  PRIMARY KEY ("championshipId","eventId"),
  FOREIGN KEY ("championshipId") REFERENCES "championship_lookup" ("championshipId"),
  FOREIGN KEY ("eventId") REFERENCES "itinerary_event" ("eventId")
);
CREATE TABLE "championship_events" (
  "categories" TEXT,
  "clerkOfTheCourse" TEXT,
  "country.countryId" INTEGER,
  "country.iso2" TEXT,
  "country.iso3" TEXT,
  "country.name" TEXT,
  "countryId" INTEGER,
  "eventId" INTEGER PRIMARY KEY,
  "finishDate" TEXT,
  "location" TEXT,
  "mode" TEXT,
  "name" TEXT,
  "organiserUrl" TEXT,
  "slug" TEXT,
  "startDate" TEXT,
  "stewards" TEXT,
  "surfaces" TEXT,
  "templateFilename" TEXT,
  "timeZoneId" TEXT,
  "timeZoneName" TEXT,
  "timeZoneOffset" INTEGER,
  "trackingEventId" INTEGER ,
  FOREIGN KEY ("eventId") REFERENCES "itinerary_event" ("eventId")
);
CREATE TABLE "championship_entries_drivers" (
  "championshipEntryId" INTEGER PRIMARY KEY ,
  "championshipId" INTEGER,
  "entrantId" TEXT,
  "ManufacturerTyre" TEXT,
  "Manufacturer" TEXT,
  "FirstName" TEXT,
  "CountryISO3" TEXT,
  "CountryISO2" TEXT,
  "LastName" TEXT,
  "manufacturerId" INTEGER,
  "personId" INTEGER,
  "tyreManufacturer" TEXT,
  FOREIGN KEY ("championshipId") REFERENCES "championship_lookup" ("championshipId")
);
CREATE TABLE "event_metadata" (
  "_id" TEXT,
  "availability" TEXT,
  "date-finish" TEXT,
  "date-start" TEXT,
  "gallery" TEXT,
  "hasdata" TEXT,
  "hasfootage" TEXT,
  "hasvideos" TEXT,
  "id" TEXT,
  "info-based" TEXT,
  "info-categories" TEXT,
  "info-date" TEXT,
  "info-flag" TEXT,
  "info-surface" TEXT,
  "info-website" TEXT,
  "kmlfile" TEXT,
  "logo" TEXT,
  "name" TEXT,
  "org-website" TEXT,
  "poi-Klo im Wald" TEXT,
  "poilistid" TEXT,
  "position" TEXT,
  "rosterid" TEXT,
  "sas-eventid" TEXT,
  "sas-itineraryid" TEXT,
  "sas-rallyid" TEXT,
  "sas-trackingid" TEXT,
  "sitid" TEXT,
  "testid" TEXT,
  "thumbnail" TEXT,
  "time-zone" TEXT,
  "tzoffset" TEXT,
  "year" INTEGER
);


'''


#conn = sqlite3.connect('wrc18_test1keys.db')
#c = conn.cursor()
#c.executescript(setup_q)

In [98]:
#Maybe bring in additional tables 
#eg via https://blog.ouseful.info/2016/11/14/what-nationality-did-you-say-you-were-again/
#bring in a table to give nationalites from country codes
#Or maybe make that a pip package?

In [99]:
setup_views_q = '''
'''

In [100]:
#meta={'rallyId':None, 'stages':[], 'championshipId':None }

In [324]:
def _getEventMetadata():
    ''' Get event metadata as JSON data feed from WRC API. '''
    url='https://webappsdata.wrc.com/srv/wrc/json/api/wrcsrv/byType?t=%22Event%22&maxdepth=1'
    eventmeta = requests.get(url).json()
    return eventmeta

def getEventMetadata():
    ''' Get a list of events from WRC as a flat pandas dataframe.
        Itinerary / event data is only available for rallies starting in stated year. '''
    eventMetadata = json_normalize(_getEventMetadata(),
                                   record_path='_meta',
                                   meta='_id'  ).drop_duplicates().pivot('_id', 'n','v').reset_index()

    eventMetadata['date-finish']=pd.to_datetime(eventMetadata['date-finish'])
    eventMetadata['date-start']=pd.to_datetime(eventMetadata['date-start'])
    eventMetadata['year'] = eventMetadata['date-start'].dt.year
    
    return eventMetadata


getEventMetadata().head()

n,_id,availability,date-finish,date-start,gallery,hasdata,hasvideos,id,info-based,info-categories,...,sas-eventid,sas-itineraryid,sas-rallyid,sas-trackingid,sitid,testid,thumbnail,time-zone,tzoffset,year
0,029e41e3-afff-44ad-950f-9af3dccf06be,now,2015-08-02,2015-07-30,NaN,NaN,true,neste-oil-rally-finland,Jyväskylä,"WRC, WRC-2, WRC-3, JWRC",...,NaN,NaN,NaN,NaN,143,NaN,NaN,Helsinki,NaN,2015
1,03bea1af-87f3-43d2-b499-0d9095440ee5,NaN,2019-08-25,2019-08-22,NaN,NaN,NaN,NaN,Bostasee,NaN,...,87,138,103,2791,87,NaN,NaN,NaN,0,2019
2,03d002d1-82d8-499c-8252-bdc36685ffd3,now,2015-10-25,2015-10-22,NaN,NaN,true,rallyracc-rally-de-espana,Salou,"WRC, WRC-2, WRC-3",...,NaN,NaN,NaN,NaN,151,NaN,NaN,Madrid,NaN,2015
3,0919ef82-a65f-4115-8372-1364be04a219,now,2015-10-04,2015-10-02,NaN,NaN,true,rallye-de-france-alsace,Corte,"WRC, WRC-2, WRC-3, JWRC",...,NaN,NaN,NaN,NaN,149,NaN,NaN,Paris,NaN,2015
4,098ea36b-902d-4292-8e3a-c85b6ae741b6,NaN,2019-03-10,2019-03-07,NaN,NaN,NaN,NaN,Leon,NaN,...,80,131,96,2784,80,NaN,NaN,NaN,-25200000,2019


In [102]:
def _getRallyIDs2(year=YEAR):
    em=getEventMetadata()
    em = em[em['year']==year][['name','sas-rallyid', 'kmlfile', 'date-start']].reset_index(drop=True).dropna()
    em['stub']=em['kmlfile'].apply(lambda x: x.split('_')[0])
    return em

def getRallyIDs2(year=YEAR):
    em = _getRallyIDs2(year=year)
    return em[['stub','sas-rallyid']].set_index('stub').to_dict()['sas-rallyid']

def listRallies2(year=YEAR):
    return getRallyIDs2(year)

In [103]:
listRallies2()

{'montecarlo': '94'}

In [104]:
listRallies2()

{'montecarlo': '94'}

In [105]:
#rallyIDs = getRallyIDs2()
#rallyIDs

In [106]:
def nvToDict(nvdict, key='n',val='v', retdict=None):
    if retdict is None:
        retdict={nvdict[key]:nvdict[val]}
    else:
        retdict[nvdict[key]]=nvdict[val]
    return retdict
#assert nvToDict({'n': "id",'v': "adac-rallye-deutschland"}) == {'id': 'adac-rallye-deutschland'}

In [107]:
 #getEventMetadata()['rosterid'].iloc[0]

In [108]:
'''
def set_rallyId(rally, year, rallyIDs=None):
    meta={'rallyId':None, 'stages':[], 'championshipId':None }
    if rallyIDs is None:
        rallyIDs = getRallyIDs()
    if rally in rallyIDs:
        meta['rallyId']=rallyIDs[rally]
        meta['rally_name'] = rally
    return meta
'''

def set_rallyId2(rally, year, rallyIDs=None):
    meta={'rallyId':None, 'stages':[], 'championshipId':None }
    if rallyIDs is None:
        rallyIDs = getRallyIDs2()
    if rally in rallyIDs:
        meta['rallyId']=rallyIDs[rally]
        meta['rally_name'] = rally
    return meta

In [109]:
getEventMetadata()

n,_id,availability,date-finish,date-start,gallery,hasdata,hasvideos,id,info-based,info-categories,...,sas-eventid,sas-itineraryid,sas-rallyid,sas-trackingid,sitid,testid,thumbnail,time-zone,tzoffset,year
0,029e41e3-afff-44ad-950f-9af3dccf06be,now,2015-08-02,2015-07-30,NaN,NaN,true,neste-oil-rally-finland,Jyväskylä,"WRC, WRC-2, WRC-3, JWRC",...,NaN,NaN,NaN,NaN,143,NaN,NaN,Helsinki,NaN,2015
1,03bea1af-87f3-43d2-b499-0d9095440ee5,NaN,2019-08-25,2019-08-22,NaN,NaN,NaN,NaN,Bostasee,NaN,...,87,138,103,2791,87,NaN,NaN,NaN,0,2019
2,03d002d1-82d8-499c-8252-bdc36685ffd3,now,2015-10-25,2015-10-22,NaN,NaN,true,rallyracc-rally-de-espana,Salou,"WRC, WRC-2, WRC-3",...,NaN,NaN,NaN,NaN,151,NaN,NaN,Madrid,NaN,2015
3,0919ef82-a65f-4115-8372-1364be04a219,now,2015-10-04,2015-10-02,NaN,NaN,true,rallye-de-france-alsace,Corte,"WRC, WRC-2, WRC-3, JWRC",...,NaN,NaN,NaN,NaN,149,NaN,NaN,Paris,NaN,2015
4,098ea36b-902d-4292-8e3a-c85b6ae741b6,NaN,2019-03-10,2019-03-07,NaN,NaN,NaN,NaN,Leon,NaN,...,80,131,96,2784,80,NaN,NaN,NaN,-25200000,2019
5,0b24651f-0eeb-4e44-81ed-2a5aa3a3eba9,now,2017-04-30,2017-04-27,NaN,NaN,true,rally-argentina,Villa Carlos Paz,"WRC, WRC-2, WRC-3",...,NaN,NaN,NaN,NaN,224,NaN,NaN,Buenos Aires,-14400000,2017
6,0d6cf890-381e-4010-9fd8-36d942ac80d2,now,2017-02-12,2017-02-09,NaN,NaN,true,rally-sweden,Karlstad,"WRC, WRC-2, WRC-3",...,NaN,NaN,NaN,NaN,218,NaN,NaN,Stockholm,NaN,2017
7,0f3fa7ea-f3f1-493a-87d5-5cb3f0fad39e,now,2018-09-16,2018-09-13,NaN,NaN,true,NaN,Marmaris,NaN,...,35,114,39,2596,35,NaN,NaN,NaN,10800000,2018
8,1127f742-70ee-4dc7-b093-05d7e7a8b8fa,NaN,2019-03-31,2019-03-28,NaN,NaN,NaN,NaN,Bastia,NaN,...,81,132,97,2785,81,NaN,NaN,NaN,0,2019
9,117b1be7-1d26-447f-a405-878da1112436,now,2014-08-24,2014-08-21,NaN,NaN,true,adac-rallye-deutschland,Trier,"WRC, WRC-2, WRC-3",...,NaN,NaN,NaN,NaN,91,NaN,http://www.wrc.com/fileadmin/images/Calendar/2...,Europe/Berlin,NaN,2014


In [110]:
#roster_id='bab64d15-4691-4561-a6bf-7284f3bd85f9'
import requests
#roster_json = requests.get( '{}&maxdepth=2'.format(wrcapi.format(roster_id),) ).json()
#roster_json   

#TO CHECK - is the sas-entryid the entryid we use elsewhere?

#This comes from event metadata
def _getRoster(roster_id):
    roster_json = requests.get(wrcapi.format(roster_id) ).json()
    roster=json_normalize(roster_json)
    
    aa=json_normalize(roster_json, record_path='_dchildren')
    zz=json_normalize(roster_json['_dchildren'],record_path=['_meta'], meta='_id').pivot('_id', 'n','v').reset_index()
    zz=pd.merge(zz,aa[['_id','name','type']], on='_id')[['fiasn','filename','sas-entryid','name']]
    zz.columns = ['fiasn','code','sas-entryid','roster_num']
    #defensive?
    zz = zz.dropna(subset=['sas-entryid'])
    return zz

def getRoster(meta):
    em = getEventMetadata()
    roster_id= em[em['sas-rallyid']==meta['rallyId']]['rosterid'].iloc[0]
    return _getRoster(roster_id)

In [111]:
def getItinerary(meta):
    ''' Get event itinerary. Also updates the stages metadata. '''
    itinerary_json=requests.get( url_base.format(stub=stubs['itinerary'].format(**meta) ) ).json()
    itinerary_event = json_normalize(itinerary_json).drop('itineraryLegs', axis=1)
    
    #meta='eventId' for eventId
    itinerary_legs = json_normalize(itinerary_json, 
                                    record_path='itineraryLegs').drop('itinerarySections', axis=1)
    #meta='eventId' for eventId
    itinerary_sections = json_normalize(itinerary_json,
                                        ['itineraryLegs', 'itinerarySections']).drop(['stages','controls'],axis=1)

    itinerary_stages=json_normalize(itinerary_json['itineraryLegs'],
                                    ['itinerarySections','stages'],
                                   meta=['itineraryLegId',['itinerarySections','itinerarySectionId']])
    meta['stages']=itinerary_stages['stageId'].tolist()
    #Should do this a pandas idiomatic way
    #meta['_stages']=zip(itinerary_stages['stageId'].tolist(),
     #                   itinerary_stages['code'].tolist(),
     #                   itinerary_stages['status'].tolist())
    meta['_stages'] = itinerary_stages[['stageId','code','status']].set_index('code').to_dict(orient='index')
    itinerary_controls=json_normalize(itinerary_json['itineraryLegs'], 
                                  ['itinerarySections','controls'] ,
                                     meta=['itineraryLegId',['itinerarySections','itinerarySectionId']])
    itinerary_controls['stageId'] = itinerary_controls['stageId'].fillna(-1).astype(int)
    
    return itinerary_event, itinerary_legs, itinerary_sections, itinerary_stages, itinerary_controls

In [113]:
itinerary_json=requests.get( url_base.format(stub=stubs['itinerary'].format(**meta) ) ).json()
itinerary_json

{'itineraryLegs': [{'itinerarySections': [{'controls': [{'controlId': 3667,
       'eventId': 78,
       'stageId': None,
       'type': 'TimeControl',
       'code': 'TC0',
       'location': 'GAP (Place Desmichels)',
       'timingPrecision': 'Minute',
       'distance': 0.0,
       'targetDuration': None,
       'targetDurationMs': None,
       'firstCarDueDateTime': '2019-01-24T17:50:00',
       'firstCarDueDateTimeLocal': '2019-01-24T18:50:00+01:00',
       'status': 'ToRun',
       'controlPenalties': 'All',
       'roundingPolicy': 'NoRounding'},
      {'controlId': 3720,
       'eventId': 78,
       'stageId': 881,
       'type': 'TimeControl',
       'code': 'TC1',
       'location': 'LA BREOLE',
       'timingPrecision': 'Minute',
       'distance': 32.52,
       'targetDuration': '00:45:00',
       'targetDurationMs': 2700000,
       'firstCarDueDateTime': '2019-01-24T18:35:00',
       'firstCarDueDateTimeLocal': '2019-01-24T19:35:00+01:00',
       'status': 'ToRun',
       

In [114]:
'''
{'rallyId': '30',
 'stages': [],
 'championshipId': None,
 'rally_name': 'montecarlo'}
 '''

#https://www.wrc.com/service/sasCacheApi.php?route=events%2F78%2Frallies%2F94%2Fitinerary
meta

{'rallyId': '94',
 'stages': [881,
  885,
  894,
  886,
  888,
  889,
  892,
  887,
  895,
  893,
  891,
  890,
  884,
  883,
  882,
  880],
 'championshipId': None,
 'rally_name': 'montecarlo',
 '_stages': {'SS1': {'stageId': 881, 'status': 'ToRun'},
  'SS2': {'stageId': 885, 'status': 'ToRun'},
  'SS3': {'stageId': 894, 'status': 'ToRun'},
  'SS4': {'stageId': 886, 'status': 'ToRun'},
  'SS5': {'stageId': 888, 'status': 'ToRun'},
  'SS6': {'stageId': 889, 'status': 'ToRun'},
  'SS7': {'stageId': 892, 'status': 'ToRun'},
  'SS8': {'stageId': 887, 'status': 'ToRun'},
  'SS9': {'stageId': 895, 'status': 'ToRun'},
  'SS10': {'stageId': 893, 'status': 'ToRun'},
  'SS11': {'stageId': 891, 'status': 'ToRun'},
  'SS12': {'stageId': 890, 'status': 'ToRun'},
  'SS13': {'stageId': 884, 'status': 'ToRun'},
  'SS14': {'stageId': 883, 'status': 'ToRun'},
  'SS15': {'stageId': 882, 'status': 'ToRun'},
  'SS16': {'stageId': 880, 'status': 'ToRun'}}}

In [115]:
getItinerary(meta)

(   eventId  itineraryId       name  priority
 0       78          144  Itinerary         1,
    itineraryId  itineraryLegId     legDate                   name  order  \
 0          144             155  2019-01-24  Thursday 24th January      1   
 1          144             154  2019-01-25    Friday 25th January      2   
 2          144             153  2019-01-26  Saturday 26th January      3   
 3          144             152  2019-01-27    Sunday 27th January      4   
 
    startListId status  
 0        255.0  ToRun  
 1          NaN  ToRun  
 2          NaN  ToRun  
 3          NaN  ToRun  ,
    itineraryLegId  itinerarySectionId       name  order
 0             155                 362  Section 1      1
 1             154                 361  Section 2      2
 2             154                 360  Section 3      3
 3             153                 359  Section 4      4
 4             153                 358  Section 5      5
 5             152                 363  Section 6   

In [116]:
#a,b,c,d,e = getItinerary(meta)

In [117]:
def _get_single_json_table(meta, stub):
    _json = requests.get( url_base.format(stub=stubs[stub].format(**meta) ) ).json()
    return json_normalize(_json)

In [118]:
#meta =  set_rallyId(name, year)

#startlists_json=requests.get( url_base.format(stub=stubs['startlists'].format(**meta) ) ).json()
#ff=[]
#for f in startlists_json:
#    if f['manufacturer']['logoFilename'] is None:
#        f['manufacturer']['logoFilename']=''
#    if f['entrant']['logoFilename'] is None:
#        f['entrant']['logoFilename']='' 
#    ff.append(f)
#ff

In [119]:
#startlists = json_normalize(ff).drop('eventClasses', axis=1)

In [120]:
def get_startlists(meta):
    startlists_json=requests.get( url_base.format(stub=stubs['startlists'].format(**meta) ) ).json()
    ff=[]
    for f in startlists_json:
        if f['manufacturer']['logoFilename'] is None:
            f['manufacturer']['logoFilename']=''
        if f['entrant']['logoFilename'] is None:
            f['entrant']['logoFilename']='' 
        ff.append(f)
    startlists = json_normalize(ff).drop('eventClasses', axis=1)
    startlist_classes = json_normalize(ff,['eventClasses'], 'entryId' )
    #startlists = json_normalize(startlists_json).drop('eventClasses', axis=1)
    #startlist_classes = json_normalize(startlists_json,['eventClasses'], 'entryId' )
    
    return startlists, startlist_classes 

In [121]:
def get_penalties(meta):
    ''' Get the list of penalties for a specified event. '''
    penalties = _get_single_json_table(meta, 'penalties')
    return penalties

In [122]:
def get_retirements(meta):
    ''' Get the list of retirements for a specified event. '''
    retirements = _get_single_json_table(meta, 'retirements')
    return retirements

In [123]:
def get_stagewinners(meta):
    ''' Get the stage winners table for a specified event. '''
    stagewinners = _get_single_json_table(meta, 'stagewinners')
    return stagewinners

In [124]:
def _single_stage(meta2, stub, stageId):
    ''' For a single stageId, get the requested resource. '''
    meta2['stageId']=stageId
    _json=requests.get( url_base.format(stub=stubs[stub].format(**meta2) ) ).json()
    _df = json_normalize(_json)
    _df['stageId'] = stageId
    return _df

def _stage_iterator(meta, stub, stage=None):
    ''' Iterate through a list of stageId values and get requested resource. '''
    meta2={'rallyId':meta['rallyId']}
    df = pd.DataFrame()
    #If stage is None get data for all stages
    if stage is not None:
        stages=[]
        #If we have a single stage (specified in form SS4) get it
        if isinstance(stage,str) and stage in meta['_stages']:
            stages.append(meta['_stages'][stage]['stageId'])
        #If we have a list of stages (in form ['SS4','SS5']) get them all
        elif isinstance(stage, list):
            for _stage in stage:
                if isinstance(_stage,str) and _stage in meta['_stages']:
                    stages.append(meta['_stages'][_stage]['stageId'])
                elif _stage in meta['stages']:
                    stages.append(_stage)
    else:
        stages = meta['stages']
        
    #Get data for required stages
    for stageId in stages:
        #meta2['stageId']=stageId
        #_json=requests.get( url_base.format(stub=stubs[stub].format(**meta2) ) ).json()
        #_df = json_normalize(_json)
        #_df['stageId'] = stageId
        _df = _single_stage(meta2, stub, stageId)
        df = pd.concat([df, _df], sort=False)
    return df.reset_index(drop=True)

In [125]:
def get_overall(meta, stage=None):
    ''' Get the overall results table for all stages on an event or a specified stage. '''
    stage_overall = _stage_iterator(meta, 'overall', stage)
    return stage_overall

In [126]:
def get_splitTimes(meta, stage=None):
    ''' Get split times table for all stages on an event or a specified stage. '''
    split_times = _stage_iterator(meta, 'split_times', stage)
    return split_times

In [127]:
def get_stage_times_stage(meta, stage=None):
    ''' Get stage times table for all stages on an event or a specified stage. '''
    stage_times_stage = _stage_iterator(meta, 'stage_times_stage', stage)
    return stage_times_stage

In [128]:
def get_stage_times_overall(meta,stage=None):
    ''' Get overall stage times table for all stages on an event or a specified stage. '''
    stage_times_overall = _stage_iterator(meta, 'stage_times_overall', stage)
    return stage_times_overall

In [180]:
#There must be a JSON/API way of getting this rather than having to fish for it
def _get_championship_codesOLD(url=None):
    if url is None:
        url = 'http://www.wrc.com/en/wrc/results/championship-standings/page/4176----.html'
    html2=requests.get(url).text
    m = re.search("var championshipClasses = (.*?);", html2, re.DOTALL)
    mm=m.group(1).replace('\n','').replace("'",'"')
    #Hack for null table
    for v in ['jwrcDriver','jwrcCoDriver','wrcDriver','wrcCoDriver','wrcManufacturers','wrc2ProDriver',
              'wrc2ProCoDriver','wrc2ProManufacturers','wrc2Driver','wrc2CoDriver']:
        mm = mm.replace(v,'[]')
    d=json.loads(mm)
    #https://stackoverflow.com/a/35758583/454773
    championshipClasses={k.replace(' ', ''): v for k, v in d.items()}
    return championshipClasses

In [181]:
#         'championship':'seasons/4/championships/{championshipId}',
#         'championship_results':'seasons/4/championships/{championshipId}/results',
_get_championship_codesOLD(season)

In [ ]:
# TO DO
def get_championship_rounds():
    pass
    #https://www.wrc.com/service/sasCacheApi.php?route=seasons%2F4%2Fchampionships%2F24

In [271]:
def get_seasons():
    ''' Get season info. '''
    return requests.get(url_root.format(stub=stubs['seasons'] )).json()
#https://www.wrc.com/service/sasCacheApi.php?route=seasons/
get_seasons()

http://www.wrc.com/service/sasCacheApi.php?route=events/78/seasons


[{'seasonId': 1, 'name': 'World Rally Championship', 'year': 2018},
 {'seasonId': 4, 'name': 'World Rally Championship', 'year': 2019}]

In [273]:
def getSeasonDetails(seasonId):
    return requests.get(url_root.format(stub=stubs['seasonDetails'].format(seasonId=seasonId) )).json()
getSeasonDetails(4)

{'championships': [{'championshipId': 24,
   'seasonId': 4,
   'name': 'FIA World Rally Championship for Drivers',
   'type': 'Person',
   'fieldOneDescription': 'FirstName',
   'fieldTwoDescription': 'LastName',
   'fieldThreeDescription': 'CountryISO3',
   'fieldFourDescription': 'Manufacturer',
   'fieldFiveDescription': 'TyreManufacturer'},
  {'championshipId': 25,
   'seasonId': 4,
   'name': 'FIA World Rally Championship for Co-Drivers',
   'type': 'Person',
   'fieldOneDescription': 'FirstName',
   'fieldTwoDescription': 'LastName',
   'fieldThreeDescription': 'CountryISO3',
   'fieldFourDescription': 'Manufacturer',
   'fieldFiveDescription': 'TyreManufacturer'},
  {'championshipId': 26,
   'seasonId': 4,
   'name': 'FIA World Rally Championship for Manufacturers',
   'type': 'Manufacturer',
   'fieldOneDescription': 'Name',
   'fieldTwoDescription': 'Manufacturer',
   'fieldThreeDescription': 'LogoFileName',
   'fieldFourDescription': '',
   'fieldFiveDescription': ''},
  {'ch

In [337]:
# TO DO set seasonId properly
def championship_tables(champ_class=None, champ_typ=None, seasonId=4):
    ''' Get all championship tables in a particular championship and / or class. '''
    #if championship is None then get all
    championship_lookup = pd.DataFrame()
    championship_entries_all = {}
    championship_rounds = pd.DataFrame()
    championship_events = pd.DataFrame()
    championship_results = pd.DataFrame()
    
    championships = getSeasonDetails(seasonId)['championships']
    
    for championship in championships:
        champ_num = championship['championshipId']
        #TO DO - are we setting the champType correctly?
        # championship['type'] returns as Person or Manufacturer
        champType = championship['name'].split()[-1]#championship['type']
        if champType not in championship_entries_all:
            championship_entries_all[champType] = pd.DataFrame()
            
        meta2={'championshipId': champ_num}
        championship_url = url_root.format(stub=stubs['championship'].format(**meta2) )
        championship_json=requests.get( championship_url ).json()
        if championship_json:
            _championship_lookup = json_normalize(championship_json).drop(['championshipEntries','championshipRounds'], axis=1)
            _championship_lookup['_codeClass'] = championship['name']
            _championship_lookup['_codeTyp'] = championship['type']
            championship_lookup = pd.concat([championship_lookup,_championship_lookup],sort=True)
    
            championships={}
            championship_dict = _championship_lookup.to_dict()
            championships[champ_num] = {c:championship_dict[c][0] for c in championship_dict}
            renamer={c.replace('Description',''):championships[champ_num][c] for c in championships[champ_num] if c.startswith('field')}            
            _championship_entries = json_normalize(championship_json,['championshipEntries'] )
            _championship_entries = _championship_entries.rename(columns=renamer)
            _championship_entries = _championship_entries[[c for c in _championship_entries.columns if c!='']]
            #pd.concat sort=False to retain current behaviour
            
            championship_entries_all[champType] = pd.concat([championship_entries_all[champType],_championship_entries],sort=False)

            _championship_rounds = json_normalize(championship_json,['championshipRounds'] ).drop('event', axis=1)
            championship_rounds = pd.concat([championship_rounds,_championship_rounds],sort=False).drop_duplicates()

            _events_json = json_normalize(championship_json,['championshipRounds' ])['event']
            _championship_events = json_normalize(_events_json)
            #Below also available as eg https://www.wrc.com/service/sasCacheApi.php?route=seasons/4/championships/24
            championship_events = pd.concat([championship_events,_championship_events],sort=False).drop_duplicates()

            _championship_results = _get_single_json_table(meta2, 'championship_results')
            championship_results = pd.concat([championship_results, _championship_results],sort=False)
    
    for k in championship_entries_all:
        championship_entries_all[k].reset_index(drop=True)
        if k in ['Driver', 'Co-Driver']:
            championship_entries_all[k] = championship_entries_all[k].rename(columns={'TyreManufacturer':'ManufacturerTyre'})
    
    return championship_lookup.reset_index(drop=True), \
            championship_results.reset_index(drop=True), \
            championship_entries_all, \
            championship_rounds.reset_index(drop=True), \
            championship_events.reset_index(drop=True)

#championship_tables()

## Usage

In [133]:
#listRallies2()

In [190]:
def cleardbtable(conn, table):
    ''' Clear the table whilst retaining the table definition '''
    c = conn.cursor()
    c.execute('DELETE FROM "{}"'.format(table))
    
def dbfy(conn, df, table, if_exists='upsert', index=False, clear=False, **kwargs):
    ''' Save a dataframe as a SQLite table.
        Clearing or replacing a table will first empty the table of entries but retain the structure. '''
    if if_exists=='replace':
        clear=True
        if_exists='append'
    if clear: cleardbtable(conn, table)
    if if_exists=='upsert':
        DB[table].upsert_all(df.to_dict(orient='records'))
    else:
        df.to_sql(table,conn,if_exists=if_exists,index=index)

In [351]:
def save_rally(meta, conn, stage=None):
    ''' Save all tables associated with a particular rally. '''
    
    if stage is None:
        
        roster = getRoster(meta)
        dbfy(conn, roster, 'roster', if_exists='replace')

        itinerary_event, itinerary_legs, itinerary_sections, \
        itinerary_stages, itinerary_controls = getItinerary(meta)

        dbfy(conn, itinerary_event, 'itinerary_event', if_exists='replace')
        dbfy(conn, itinerary_legs, 'itinerary_legs', if_exists='replace')
        dbfy(conn, itinerary_sections, 'itinerary_sections', if_exists='replace')
        dbfy(conn, itinerary_stages, 'itinerary_stages', if_exists='replace')
        dbfy(conn, itinerary_controls, 'itinerary_controls', if_exists='replace')

        startlists, startlist_classes = get_startlists(meta)
        dbfy(conn, startlists, 'startlists', if_exists='replace')
        dbfy(conn, startlist_classes, 'startlist_classes', if_exists='replace')

    #These need to be upserted
    penalties = get_penalties(meta)
    dbfy(conn, penalties, 'penalties')

    retirements = get_retirements(meta)
    dbfy(conn, retirements, 'retirements')

    stagewinners = get_stagewinners(meta)
    dbfy(conn, stagewinners, 'stagewinners')


    stage_overall = get_overall(meta, stage)
    dbfy(conn, stage_overall, 'stage_overall')

    split_times = get_splitTimes(meta, stage)
    dbfy(conn, split_times, 'split_times')
    
    stage_times_stage = get_stage_times_stage(meta, stage)
    dbfy(conn, stage_times_stage, 'stage_times_stage')
    
    stage_times_overall = get_stage_times_overall(meta, stage)
    dbfy(conn, stage_times_overall, 'stage_times_overall')

    
def save_championship(meta, conn):
    ''' Save all championship tables for a particular year. '''
    championship_lookup, championship_results, _championship_entries_all, \
        championship_rounds, championship_events = championship_tables()
        
    championship_entries_drivers = _championship_entries_all['Drivers']
    championship_entries_codrivers = _championship_entries_all['Co-Drivers']
    championship_entries_manufacturers = _championship_entries_all['Manufacturers']
    #championship_entries_nations = _championship_entries_all['Nations']
    
    dbfy(conn, championship_lookup, 'championship_lookup', if_exists='replace')
    dbfy(conn, championship_results, 'championship_results', if_exists='replace')
    dbfy(conn, championship_entries_drivers, 'championship_entries_drivers',if_exists='replace')
    dbfy(conn, championship_entries_codrivers, 'championship_entries_codrivers', if_exists='replace')
    dbfy(conn, championship_entries_manufacturers, 'championship_entries_manufacturers', if_exists='replace')
    dbfy(conn, championship_rounds, 'championship_rounds', if_exists='replace')
    dbfy(conn, championship_events, 'championship_events', if_exists='replace')

def get_one(rally, stage, dbname='wrc19_test1.db', year=YEAR):
    conn = sqlite3.connect(dbname)
    meta =  set_rallyId2(rally, year)
    getItinerary(meta) #to update meta
    print(meta)
    save_rally(meta, conn, stage)
    
def get_all(rally, dbname='wrc19_test1.db', year=YEAR):
    
    conn = sqlite3.connect(dbname)
    
    meta =  set_rallyId2(rally, year)
    
    save_rally(meta, conn)
    save_championship(meta, conn)
    
def get_championship(rally, dbname='wrc19_test1.db', year=YEAR):
    
    conn = sqlite3.connect(dbname)
    
    meta =  set_rallyId2(rally, year)

    save_championship(meta, conn)


In [340]:
championship_lookup, championship_results, _championship_entries_all, \
        championship_rounds, championship_events = championship_tables()

In [344]:
_championship_entries_all['Manufacturers']

,championshipEntryId,championshipId,entrantId,Name,LogoFileName,Manufacturer,manufacturerId,personId,tyreManufacturer
0,406,26,NaN,CITROEN TOTAL WRT,citroen,Citroen,13,None,None
1,421,26,94.0,M-SPORT FORD WORLD RALLY TEAM,ford,Ford,26,None,None
2,422,26,1.0,HYUNDAI SHELL MOBIS WRT,hyundai,Hyundai,33,None,None
3,423,26,91.0,TOYOTA GAZOO RACING WRT,toyota,Toyota,84,None,None
0,433,29,94.0,M-SPORT FORD WORLD RALLY TEAM,ford,Ford,26,None,None
1,434,29,92.0,SKODA MOTORSPORT,skoda,Skoda,77,None,None


In [136]:
#listRallies()
listRallies2()

{'montecarlo': '94'}

In [297]:
#dbname='wrc18.db'
year = 2019
name = 'montecarlo'
dbname='montecarlo19.db'

In [298]:
meta =  set_rallyId2(name, year)
meta

{'rallyId': '94',
 'stages': [],
 'championshipId': None,
 'rally_name': 'montecarlo'}

In [299]:
import os

#For some reason, these don't seem to get set / picked up correctly from a notebook?
os.environ["WRC_RESULTS_NAME"] = name
os.environ["WRC_RESULTS_DBNAME"] = dbname
os.environ["WRC_RESULTS_YEAR"] = str(year)

In [300]:
#championship_tables()

In [301]:
#set_rallyId2('uk', 2018)

In [302]:
#rr=get_retirements(meta)
#rr.head()

In [303]:
# TO DO - ability to top up just the stage we need

In [304]:
#set_rallyId("Finland",2018)
#set_rallyId2("australia",2018)

In [305]:
getEventMetadata().columns

Index(['_id', 'availability', 'date-finish', 'date-start', 'gallery',
       'hasdata', 'hasvideos', 'id', 'info-based', 'info-categories',
       'info-date', 'info-flag', 'info-surface', 'info-website', 'kmlfile',
       'logo', 'name', 'org-website', 'poi-Klo im Wald', 'poilistid',
       'position', 'rosterid', 'sas-eventid', 'sas-itineraryid', 'sas-rallyid',
       'sas-trackingid', 'sitid', 'testid', 'thumbnail', 'time-zone',
       'tzoffset', 'year'],
      dtype='object', name='n')

In [306]:
#full run
#dbname = 'wrc18.db'

In [307]:
#from sqlite_utils import Database
#DB = Database(sqlite3.connect(dbname))

In [322]:
?dbfy

In [353]:
#new db
!mv $dbname old-$dbname
!rm $dbname
conn = sqlite3.connect(dbname, timeout=10)
c = conn.cursor()

DB = Database(sqlite3.connect(dbname))

c.executescript(setup_q)
c.executescript(setup_views_q)
q="SELECT name FROM sqlite_master WHERE type = 'table';"

#The upsert breaks with the - and space chars in column names
dbfy(conn, getEventMetadata(), 'event_metadata', if_exists='replace')

pd.read_sql(q,conn)

rm: montecarlo19.db: No such file or directory


/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:2130: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype)


,name
0,itinerary_event
1,itinerary_legs
2,itinerary_sections
3,itinerary_stages
4,itinerary_controls
5,startlists
6,roster
7,startlist_classes
8,penalties
9,retirements


In [354]:
getEventMetadata()

n,_id,availability,date-finish,date-start,gallery,hasdata,hasvideos,id,info-based,info-categories,...,sas-eventid,sas-itineraryid,sas-rallyid,sas-trackingid,sitid,testid,thumbnail,time-zone,tzoffset,year
0,029e41e3-afff-44ad-950f-9af3dccf06be,now,2015-08-02,2015-07-30,NaN,NaN,true,neste-oil-rally-finland,Jyväskylä,"WRC, WRC-2, WRC-3, JWRC",...,NaN,NaN,NaN,NaN,143,NaN,NaN,Helsinki,NaN,2015
1,03bea1af-87f3-43d2-b499-0d9095440ee5,NaN,2019-08-25,2019-08-22,NaN,NaN,NaN,NaN,Bostasee,NaN,...,87,138,103,2791,87,NaN,NaN,NaN,0,2019
2,03d002d1-82d8-499c-8252-bdc36685ffd3,now,2015-10-25,2015-10-22,NaN,NaN,true,rallyracc-rally-de-espana,Salou,"WRC, WRC-2, WRC-3",...,NaN,NaN,NaN,NaN,151,NaN,NaN,Madrid,NaN,2015
3,0919ef82-a65f-4115-8372-1364be04a219,now,2015-10-04,2015-10-02,NaN,NaN,true,rallye-de-france-alsace,Corte,"WRC, WRC-2, WRC-3, JWRC",...,NaN,NaN,NaN,NaN,149,NaN,NaN,Paris,NaN,2015
4,098ea36b-902d-4292-8e3a-c85b6ae741b6,NaN,2019-03-10,2019-03-07,NaN,NaN,NaN,NaN,Leon,NaN,...,80,131,96,2784,80,NaN,NaN,NaN,-25200000,2019
5,0b24651f-0eeb-4e44-81ed-2a5aa3a3eba9,now,2017-04-30,2017-04-27,NaN,NaN,true,rally-argentina,Villa Carlos Paz,"WRC, WRC-2, WRC-3",...,NaN,NaN,NaN,NaN,224,NaN,NaN,Buenos Aires,-14400000,2017
6,0d6cf890-381e-4010-9fd8-36d942ac80d2,now,2017-02-12,2017-02-09,NaN,NaN,true,rally-sweden,Karlstad,"WRC, WRC-2, WRC-3",...,NaN,NaN,NaN,NaN,218,NaN,NaN,Stockholm,NaN,2017
7,0f3fa7ea-f3f1-493a-87d5-5cb3f0fad39e,now,2018-09-16,2018-09-13,NaN,NaN,true,NaN,Marmaris,NaN,...,35,114,39,2596,35,NaN,NaN,NaN,10800000,2018
8,1127f742-70ee-4dc7-b093-05d7e7a8b8fa,NaN,2019-03-31,2019-03-28,NaN,NaN,NaN,NaN,Bastia,NaN,...,81,132,97,2785,81,NaN,NaN,NaN,0,2019
9,117b1be7-1d26-447f-a405-878da1112436,now,2014-08-24,2014-08-21,NaN,NaN,true,adac-rallye-deutschland,Trier,"WRC, WRC-2, WRC-3",...,NaN,NaN,NaN,NaN,91,NaN,http://www.wrc.com/fileadmin/images/Calendar/2...,Europe/Berlin,NaN,2014


In [296]:
#DEBUG
#for example: OperationalError: table event_metadata has no column named hasfootage
#

In [186]:
#itinerary_event

In [187]:
#itinerary_event

In [355]:
get_championship(name, dbname=dbname, year=year )

In [356]:
#set_rallyId( name,year )
get_all(name, dbname=dbname, year=year )

In [99]:
#full run:
for name in listRallies2():
    print('trying {}'.format(name))
    get_all(name, dbname=dbname, year=year )

trying turkey
trying sardegna
trying france
trying spain
trying finland
trying portugal
trying germany
trying mexico
trying argentina
trying australia
trying montecarlo
trying uk
trying sweden


In [249]:
#PK issues?
#upsert issues? Pandas doesn't support upsert

#Use Simon Willison's sqlite utils, which has upsert.
get_one(name, 'SS4', dbname=dbname, year=year)

{'rallyId': '94', 'stages': [881, 885, 894, 886, 888, 889, 892, 887, 895, 893, 891, 890, 884, 883, 882, 880], 'championshipId': None, 'rally_name': 'montecarlo', '_stages': {'SS1': {'stageId': 881, 'status': 'Completed'}, 'SS2': {'stageId': 885, 'status': 'Completed'}, 'SS3': {'stageId': 894, 'status': 'Cancelled'}, 'SS4': {'stageId': 886, 'status': 'Running'}, 'SS5': {'stageId': 888, 'status': 'Running'}, 'SS6': {'stageId': 889, 'status': 'ToRun'}, 'SS7': {'stageId': 892, 'status': 'ToRun'}, 'SS8': {'stageId': 887, 'status': 'ToRun'}, 'SS9': {'stageId': 895, 'status': 'ToRun'}, 'SS10': {'stageId': 893, 'status': 'ToRun'}, 'SS11': {'stageId': 891, 'status': 'ToRun'}, 'SS12': {'stageId': 890, 'status': 'ToRun'}, 'SS13': {'stageId': 884, 'status': 'ToRun'}, 'SS14': {'stageId': 883, 'status': 'ToRun'}, 'SS15': {'stageId': 882, 'status': 'ToRun'}, 'SS16': {'stageId': 880, 'status': 'ToRun'}}}


In [203]:
meta =  set_rallyId2(name, year)
meta

{'rallyId': '94',
 'stages': [],
 'championshipId': None,
 'rally_name': 'montecarlo'}

In [219]:
conn = sqlite3.connect(dbname)

q="SELECT name FROM sqlite_master WHERE type = 'table';"
pd.read_sql(q,conn)

,name
0,itinerary_event
1,itinerary_legs
2,itinerary_sections
3,itinerary_stages
4,itinerary_controls
5,startlists
6,roster
7,startlist_classes
8,penalties
9,retirements


In [221]:
q="SELECT * FROM event_metadata LIMIT 1;"
pd.read_sql(q,conn).columns

Index(['_id', 'availability', 'date-finish', 'date-start', 'gallery',
       'hasdata' TEXT,\n  'hasvideos', 'id', 'info-based', 'info-categories',
       'info-date', 'info-flag', 'info-surface', 'info-website', 'kmlfile',
       'logo', 'name', 'org-website', 'poi-Klo im Wald', 'poilistid',
       'position', 'rosterid', 'sas-eventid', 'sas-itineraryid', 'sas-rallyid',
       'sas-trackingid', 'sitid', 'testid', 'thumbnail', 'time-zone',
       'tzoffset', 'year'],
      dtype='object')

In [250]:
q="SELECT * FROM championship_events LIMIT 1;"
pd.read_sql(q,conn)

,categories,clerkOfTheCourse,country.countryId,country.iso2,country.iso3,country.name,countryId,eventId,finishDate,location,...,organiserUrl,slug,startDate,stewards,surfaces,templateFilename,timeZoneId,timeZoneName,timeZoneOffset,trackingEventId


In [251]:
q="SELECT * FROM championship_rounds LIMIT 1;"
pd.read_sql(q,conn)

,championshipId,eventId,order


In [ ]:
#!rm wrc18_test1.db

In [245]:
#!pip3 install isodate
import isodate
pd.to_timedelta(isodate.parse_duration('PT1H5M26S')), isodate.parse_duration('PT0.1S')
#ISO 8601 https://stackoverflow.com/questions/51168022/what-does-pt-prefix-stand-for-in-duration

# TO DO - some function that timifies particular columns

def _totime(s):
    if s and isinstance(s,str):
        if s.startswith('PT'):
            return pd.to_timedelta(isodate.parse_duration(s))
        else:
            #Should regex to check this?
            return pd.to_timedelta(s)
            
def timeify(df):
    time_cols = ['diffFirst', 'diffPrev']
    for c in [c for c in df.columns if c in time_cols]:
        df[c] = df[c].apply(_totime)
    return df

In [246]:
#Data2Text - Stage Result notebook has time wrangling

In [247]:
#Why does this get cast to time but stage_times_overall doesn't?
q="SELECT * FROM stage_times_stage LIMIT 10 ;"
timeify(pd.read_sql(q,conn)).dtypes

#The diffFirst etc are not time objects - they;re strings; cast to timedelta? DOes SQLIte do timedelta?
#The cast is easy in Python: pd.to_timedelta('00:04:45.5000000')

diffFirst            timedelta64[ns]
diffFirstMs                    int64
diffPrev             timedelta64[ns]
diffPrevMs                     int64
elapsedDuration               object
elapsedDurationMs              int64
entryId                        int64
position                       int64
source                        object
stageId                        int64
stageTimeId                    int64
status                        object
dtype: object

In [223]:
## Are stage_times_overall and stage_overall the same?


#Need to parse these time things correctly...
#Maybe parse 'PT10M40.4S'  to  '00:10:40.4000000',  'PT1H9M30.8S' to '00:01:09:30.800000'
#0.1S ->00:00:00.100000, 1M0.1s->00:10:00.100000, 1H1M0.1S ->01:10:00.100000
q="SELECT * FROM stage_times_overall ORDER BY totalTimeMs LIMIT 10 ;"
pd.read_sql(q,conn)

,diffFirst,diffFirstMs,diffPrev,diffPrevMs,entryId,penaltyTime,penaltyTimeMs,position,stageTime,stageTimeMs,totalTime,totalTimeMs,stageId
0,PT0S,0,PT0S,0,3444,PT0S,0,1,PT13M2S,782000,PT13M2S,782000,881
1,PT5S,5000,PT5S,5000,3451,PT0S,0,2,PT13M7S,787000,PT13M7S,787000,881
2,PT10.6S,10600,PT5.6S,5600,3442,PT0S,0,3,PT13M12.6S,792600,PT13M12.6S,792600,881
3,PT23.2S,23200,PT12.6S,12600,3450,PT0S,0,4,PT13M25.2S,805200,PT13M25.2S,805200,881
4,PT26.8S,26800,PT3.6S,3600,3443,PT0S,0,5,PT13M28.8S,808800,PT13M28.8S,808800,881
5,PT32.3S,32300,PT5.5S,5500,3448,PT0S,0,6,PT13M34.3S,814300,PT13M34.3S,814300,881
6,PT32.7S,32700,PT0.4S,400,3446,PT0S,0,7,PT13M34.7S,814700,PT13M34.7S,814700,881
7,PT35.4S,35400,PT2.7S,2700,3445,PT0S,0,8,PT13M37.4S,817400,PT13M37.4S,817400,881
8,PT37S,37000,PT1.6S,1600,3447,PT0S,0,9,PT13M39S,819000,PT13M39S,819000,881
9,PT52.2S,52200,PT15.2S,15200,3452,PT0S,0,10,PT13M54.2S,834200,PT13M54.2S,834200,881


In [214]:
q="SELECT * FROM stage_overall ORDER BY totalTimeMs LIMIT 10;"
pd.read_sql(q,conn)

,diffFirst,diffFirstMs,diffPrev,diffPrevMs,entryId,penaltyTime,penaltyTimeMs,position,stageTime,stageTimeMs,totalTime,totalTimeMs,stageId
0,PT0S,0,PT0S,0,3444,PT0S,0,1,PT13M2S,782000,PT13M2S,782000,881
1,PT5S,5000,PT5S,5000,3451,PT0S,0,2,PT13M7S,787000,PT13M7S,787000,881
2,PT10.6S,10600,PT5.6S,5600,3442,PT0S,0,3,PT13M12.6S,792600,PT13M12.6S,792600,881
3,PT23.2S,23200,PT12.6S,12600,3450,PT0S,0,4,PT13M25.2S,805200,PT13M25.2S,805200,881
4,PT26.8S,26800,PT3.6S,3600,3443,PT0S,0,5,PT13M28.8S,808800,PT13M28.8S,808800,881
5,PT32.3S,32300,PT5.5S,5500,3448,PT0S,0,6,PT13M34.3S,814300,PT13M34.3S,814300,881
6,PT32.7S,32700,PT0.4S,400,3446,PT0S,0,7,PT13M34.7S,814700,PT13M34.7S,814700,881
7,PT35.4S,35400,PT2.7S,2700,3445,PT0S,0,8,PT13M37.4S,817400,PT13M37.4S,817400,881
8,PT37S,37000,PT1.6S,1600,3447,PT0S,0,9,PT13M39S,819000,PT13M39S,819000,881
9,PT52.2S,52200,PT15.2S,15200,3452,PT0S,0,10,PT13M54.2S,834200,PT13M54.2S,834200,881


In [229]:
q="SELECT * FROM stagewinners;"
pd.read_sql(q,conn)


,elapsedDuration,elapsedDurationMs,entryId,stageId,stageName
0,None,NaN,NaN,881,LA BREOLE - SELONNET
1,None,NaN,NaN,885,AVANÇON - NOTRE DAME DU LAUS
2,None,NaN,NaN,888,CURBANS - PIEGUT 1
3,None,NaN,NaN,886,ROUSSIEUX - LABOREL 1
4,None,NaN,NaN,894,VALDROME - SIGOTTIER 1
5,None,NaN,NaN,892,ROUSSIEUX - LABOREL 2
6,None,NaN,NaN,889,VALDROME - SIGOTTIER 2
7,None,NaN,NaN,887,CURBANS - PIEGUT 2
8,None,NaN,NaN,893,ST LEGER LES MELEZES - LA BATIE NEUVE 1
9,None,NaN,NaN,895,AGNIERES EN DEVOLUY - CORPS 1


In [227]:
!ls -al *.db

-rw-r--r--  1 ajh59  1182653967    36864 12 Feb 15:32 amagicdemo.db
-rw-r--r--  1 ajh59  1182653967   638976 19 May 16:01 argentina18.db
-rw-r--r--  1 ajh59  1182653967        0 10 Feb 18:32 database.db
-rw-r--r--  1 ajh59  1182653967  1421312 31 Jul 14:08 finland18.db
-rw-r--r--  1 ajh59  1182653967   974848 19 May 12:46 france18.db
-rw-r--r--  1 ajh59  1182653967        0  8 Jun 08:30 italy.db
-rw-r--r--  1 ajh59  1182653967  1024000 11 Jun 09:46 italy18.db
-rw-r--r--  1 ajh59  1182653967   622592 19 May 12:45 mexico18.db
-rw-r--r--  1 ajh59  1182653967   966656 19 May 12:44 monaco18.db
-rw-r--r--  1 ajh59  1182653967  1290240 20 May 14:30 portugal18.db
-rw-r--r--  1 ajh59  1182653967        0  3 Feb 20:55 rally.db
-rw-r--r--  1 ajh59  1182653967  1236992 19 May 12:45 sweden18.db
-rw-r--r--  1 ajh59  1182653967  2117632  5 Mar 21:02 wrc18.db
-rw-r--r--  1 ajh59  1182653967   954368  5 Mar 15:33 wrc18_monaco.db
-rw-r--r--  1 ajh59  1182653967   954368  5 Mar 15:31 wrc18_sweden.db
-rw-

In [133]:
import sqlite3
import pandas as pd
conn = sqlite3.connect(dbname)
c = conn.cursor()
#c.executescript(setup_q)
#c.executescript(setup_views_q)
q="SELECT name FROM sqlite_master WHERE type = 'table';"
pd.read_sql(q,conn)

,name
0,itinerary_event
1,itinerary_legs
2,itinerary_sections
3,itinerary_stages
4,itinerary_controls
5,startlists
6,startlist_classes
7,penalties
8,retirements
9,stagewinners


In [134]:
q="SELECT * FROM startlists LIMIT 1;"
pd.read_sql(q,conn).to_dict()

{'codriver.abbvName': {0: 'J. INGRASSIA'},
 'codriver.code': {0: 'ING'},
 'codriver.country.countryId': {0: 76},
 'codriver.country.iso2': {0: 'FR'},
 'codriver.country.iso3': {0: 'FRA'},
 'codriver.country.name': {0: 'France'},
 'codriver.countryId': {0: 76},
 'codriver.firstName': {0: 'Julien'},
 'codriver.fullName': {0: 'Julien INGRASSIA'},
 'codriver.lastName': {0: 'INGRASSIA'},
 'codriver.personId': {0: 521},
 'codriverId': {0: 521},
 'driver.abbvName': {0: 'S. OGIER'},
 'driver.code': {0: 'OGI'},
 'driver.country.countryId': {0: 76},
 'driver.country.iso2': {0: 'FR'},
 'driver.country.iso3': {0: 'FRA'},
 'driver.country.name': {0: 'France'},
 'driver.countryId': {0: 76},
 'driver.firstName': {0: 'Sébastien\xa0'},
 'driver.fullName': {0: 'Sébastien\xa0 OGIER'},
 'driver.lastName': {0: 'OGIER'},
 'driver.personId': {0: 2453},
 'driverId': {0: 2453},
 'eligibility': {0: 'M'},
 'entrant.entrantId': {0: 94},
 'entrant.logoFilename': {0: ''},
 'entrant.name': {0: 'M-SPORT FORD WORLD RA